<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for retrieving data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.11 and scikit-learn 1.3.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data with train and test data split](#read)<br>
&nbsp;&nbsp;[Create pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
&nbsp;&nbsp;[Forecast](#fore)<br>
**[Store the model](#saving)**<br>
**[Summary and next steps](#summary_and_next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm-watsonx-ai,
 - autoai-ts-libs,
 - scikit-learn


In [ ]:
!pip install ibm-watsonx-ai==1.0.* | tail -n 1
!pip install -U autoai-ts-libs==4.0.* | tail -n 1
!pip install scikit-learn==1.3.* | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
from ibm_watsonx_ai.helpers import DataConnection
from ibm_watsonx_ai.helpers import ContainerLocation

training_data_references = [
    DataConnection(
        data_asset_id='cd47eb59-5fd0-4eec-b2b6-27621426c462'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/2c3d57d0-a39b-4b4d-939b-6c31a3de5251/wml_data/357ad23f-7f52-469e-b557-dee8b24a4ee5/data/autoai-ts',
        model_location='auto_ml/2c3d57d0-a39b-4b4d-939b-6c31a3de5251/wml_data/357ad23f-7f52-469e-b557-dee8b24a4ee5/data/autoai-ts/model.zip',
        training_status='auto_ml/2c3d57d0-a39b-4b4d-939b-6c31a3de5251/wml_data/357ad23f-7f52-469e-b557-dee8b24a4ee5/training-status.json'
    )
)

The following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
    prediction_type='timeseries',
    prediction_columns=['Price'],
    csv_separator=',',
    holdout_size=20,
    training_data_references=training_data_references,
    training_result_reference=training_result_reference,
    timestamp_column_name=-1,
    backtest_num=4,
    pipeline_type='customized',
    customized_pipelines=['RandomForestRegressor', 'ExogenousRandomForestRegressor', 'SVM', 'ExogenousSVM', 'LocalizedFlattenEnsembler', 'DifferenceFlattenEnsembler', 'FlattenEnsembler', 'ExogenousLocalizedFlattenEnsembler', 'ExogenousDifferenceFlattenEnsembler', 'ExogenousFlattenEnsembler', 'MT2RForecaster', 'ExogenousMT2RForecaster', 'HoltWinterAdditive', 'HoltWinterMultiplicative', 'Bats', 'ARIMA', 'ARIMAX', 'ARIMAX_RSAR', 'ARIMAX_PALR', 'ARIMAX_RAR', 'ARIMAX_DMLR'],
    lookback_window=8,
    forecast_window=1,
    max_num_daub_ensembles=3,
    feature_columns=['Price'],
    future_exogenous_available=True,
    gap_len=0,
    deployment_url='https://eu-de.ml.cloud.ibm.com',
    project_id='1d07c8ff-0040-43b9-87c6-1e1fd69aed98',
    numerical_imputation_strategy=['FlattenIterative', 'Linear', 'Cubic', 'Previous']
)

## Set `n_jobs` parameter to the number of available CPUs

In [ ]:
import os, ast
CPU_NUMBER = 4
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action**: Provide the IBM Cloud apikey, For details, see [documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [ ]:
api_key = 'PUT_YOUR_APIKEY_HERE'

In [ ]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    api_key=api_key,
    url=experiment_metadata['deployment_url']
)

In [ ]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

if 'space_id' in experiment_metadata:
    client.set.default_space(experiment_metadata['space_id'])
else:
    client.set.default_project(experiment_metadata['project_id'])
    
training_data_references[0].set_client(client)



<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data with train and test data split

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [ ]:
X_train, X_test, y_train, y_test = training_data_references[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True, use_flight=True)

<a id="preview_model_to_python_code"></a>
## Create pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_ts_libs.utils.ts_pipeline import TSPipeline
from autoai_ts_libs.transforms.imputers import linear
from autoai_ts_libs.sklearn.autoai_ts_pipeline import AutoaiTSPipeline
from autoai_ts_libs.sklearn.mvp_windowed_transformed_target_estimators import (
    AutoaiWindowTransformedTargetRegressor,
)
from autoai_ts_libs.sklearn.mvp_windowed_wrapped_regressor import (
    AutoaiWindowedWrappedRegressor,
)
from sklearn.svm import SVR

#### Pipeline definition.

In [ ]:
linear = linear(missing_val_identifier=float("nan"))
autoai_windowed_wrapped_regressor = AutoaiWindowedWrappedRegressor(
    n_jobs=CPU_NUMBER, regressor=SVR()
)
autoai_ts_pipeline_0 = AutoaiTSPipeline(
    steps=[("est", autoai_windowed_wrapped_regressor)]
)
autoai_window_transformed_target_regressor = (
    AutoaiWindowTransformedTargetRegressor(
        feature_columns=[0],
        lookback_window=8,
        random_state=33,
        regressor=autoai_ts_pipeline_0,
        row_mean_center=True,
        short_name="WindowSVR",
        target_columns=[0],
    )
)
autoai_ts_pipeline = AutoaiTSPipeline(
    steps=[("windowedttr", autoai_window_transformed_target_regressor)]
)
pipeline = TSPipeline(
    steps=[
        ("linear_imputer", linear),
        (
            "<class 'autoai_ts_libs.sklearn.autoai_ts_pipeline.AutoaiTSPipeline'>",
            autoai_ts_pipeline,
        ),
    ],
    feature_columns=[0],
    target_columns=[0],
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from autoai_ts_libs.utils.metrics import get_scorer

scorer = get_scorer("neg_avg_symmetric_mean_absolute_percentage_error")

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(X_train.values, y_train.values);

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, X_test.values, y_test.values)
print(score)

In [ ]:
pipeline.predict(X_test.values)

<a id="forecast"></a>
## Forecast

Forecast the future values of targets.

In [ ]:
pipeline.predict()

### Forecasting with the new observations
Provide new observations values
For example:
 - `a,b` are both targets and features, taken from `experiment_metadata["prediction_columns"]`

```
import numpy as np

a_newObs = [12, 22, 35, 46]
b_newObs = [34, 23, 45, 34]
newObs = np.mat([a_newObs, b_newObs]).T
pipeline.predict(newObs)
```


<a id="saving"></a>
## Store the model

In this section you will learn how to store the retrained model.

In [ ]:
model_metadata = {
    client.repository.ModelMetaNames.NAME: 'P2 - Pretrained AutoAI pipeline'
}

stored_model_details = client.repository.store_model(model=pipeline, meta_props=model_metadata, experiment_metadata=experiment_metadata)

Inspect the stored model details.

In [ ]:
stored_model_details

<a id="deployment"></a>
## Create online deployment

You can use the commands below to promote the model to space and create online deployment (web service).

<a id="working_spaces"></a>
### Working with spaces

In this section you will specify a deployment space for organizing the assets for deploying and scoring the model. If you do not have an existing space, you can use [Deployment Spaces Dashboard](/ml-runtime/spaces?context=icp4data) to create a new space, following these steps:

- Click **New Deployment Space**.
- Create an empty space.
- Select Cloud Object Storage.
- Select Watson Machine Learning instance and press **Create**.
- Copy `space_id` and paste it below.

**Tip**: You can also use the API to prepare the space for your work. Learn more [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign or update space ID below.

#### Prepare online deployment

#### Test online deployment

<a id="cleanup"></a>
### Deleting deployment
You can delete the existing deployment by calling the `client.deployments.delete(deployment_id)` command.
To list the existing web services, use `client.deployments.list()`.

<a id="summary_and_next_steps"></a>
# Summary and next steps
You successfully completed this notebook!
You learned how to use AutoAI pipeline definition to train the model.
Check out our [Online Documentation](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/cpd/overview/relnotes-4.0.html) for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License. Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>

___